# PREAMBLE

NOTE: This notebook uses the `alphas_determination` branch

In [1]:
import numpy as np
import pandas as pd
from validphys.api import API
from validphys.loader import FallbackLoader

%matplotlib inline

l = FallbackLoader()

# Definition of the input

In [2]:
fitname = "240822-05-rs-n3lo-tcm"

# COMPUTATION OF $\alpha_s$

In [3]:
fit = API.fit(fit=fitname)

prior_pdf = fit.as_input()["theorycovmatconfig"]["pdf"]

common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fit.name,
    fits=[fit.name],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
)

theoryids_dict = {
    "point_prescription": {"from_": "theorycovmatconfig"},
    "theoryids": {"from_": "scale_variation_theories"},
    "theoryid": {"from_": "theory"},
    "theory": {"from_": "fit"},
    "theorycovmatconfig": {"from_": "fit"},
} | common_dict
theoryids = API.theoryids(**theoryids_dict)
theory_plus = theoryids[1].id
theory_mid = theoryids[0].id
theory_min = theoryids[2].id

# Inputs for central theory (used to construct the alphas covmat)
inps_central = dict(theoryid=theory_mid, pdf=prior_pdf, **common_dict)

# Inputs for plus theory (used to construct the alphas covmat)
inps_plus = dict(theoryid=theory_plus, pdf=prior_pdf, **common_dict)

# Inputs for minus theory prediction (used to construct the alphas covmat)
inps_minus = dict(theoryid=theory_min, pdf=prior_pdf, **common_dict)

# inputs for the computation of the prediction of the fit with cov=C+S, where S
# is computed using the inps_central, inps_plus, and inps_minus dictionaries
inps_central_fit = dict(theoryid=theory_mid, pdf={"from_": "fit"}, **common_dict)

In [ ]:
prior_theorypreds_central = API.group_result_table_no_table(**inps_central).iloc[:, 2:].mean(axis=1)

In [5]:
prior_theorypreds_plus = API.group_result_table_no_table(**inps_plus).iloc[:, 2:].mean(axis=1)

In [6]:
prior_theorypreds_minus = API.group_result_table_no_table(**inps_minus).iloc[:, 2:].mean(axis=1)

In [7]:
# maybe we scaled the covmat to account for higher order derivatives or to test
# depencence of the prior

# NOTE: this features is not used for the final result
covmat_scaling_factor = (
    fit.as_input().get("theorycovmatconfig", {}).get("rescale_alphas_covmat", 1.0)
)

In [8]:
# Get the values of alphas...
alphas_plus = API.theory_info_table(theory_db_id=theory_plus).loc["alphas"].iloc[0]
alphas_central = API.theory_info_table(theory_db_id=theory_mid).loc["alphas"].iloc[0]
alphas_min = API.theory_info_table(theory_db_id=theory_min).loc["alphas"].iloc[0]

# ... and make sure the alphas shift in both directions is symmetric
delta_alphas_plus = alphas_plus - alphas_central
delta_alphas_min = alphas_central - alphas_min
if abs(delta_alphas_min - delta_alphas_plus) > 1e-6:
    raise ValueError("alphas shifts in both directions is not symmetric")
else:
    alphas_step_size = delta_alphas_min

In [9]:
beta_tilde = np.sqrt(covmat_scaling_factor) * (alphas_step_size / np.sqrt(2)) * np.array([1, -1])
S_tilde = beta_tilde @ beta_tilde

In [10]:
delta_plus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_plus - prior_theorypreds_central
)
delta_minus = (np.sqrt(covmat_scaling_factor) / np.sqrt(2)) * (
    prior_theorypreds_minus - prior_theorypreds_central
)

beta = [delta_plus, delta_minus]
S_hat = beta_tilde @ beta

S = np.outer(delta_plus, delta_plus) + np.outer(delta_minus, delta_minus)
S = pd.DataFrame(S, index=delta_minus.index, columns=delta_minus.index)

In [ ]:
stored_covmat = pd.read_csv(
    fit.path / "tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
    index_col=[0, 1, 2],
    header=[0, 1, 2],
    sep="\t|,",
    engine="python",
).fillna(0)
storedcovmat_index = pd.MultiIndex.from_tuples(
    [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_covmat.index],
    names=["group", "dataset", "id"],
)  # make sure theoryID is an integer, same as in S
stored_covmat = pd.DataFrame(
    stored_covmat.values, index=storedcovmat_index, columns=storedcovmat_index
)
stored_covmat = stored_covmat.reindex(S.index).T.reindex(S.index)

if not np.allclose(S, stored_covmat):
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

In [ ]:
theorypreds_fit = API.group_result_table_no_table(**inps_central_fit).iloc[:, 2:]

In [ ]:
C = API.groups_covmat(
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
    theoryid={"from_": "theory"},
    theory={"from_": "fit"},
    **common_dict
)

In [22]:
# MHOU covmat saved as user uncertainties, see
# https://docs.nnpdf.science/tutorials/general_th_covmat.html At some point we
# have to allow for different sources of theory unceratinty to be stored in
# different csv files, without this kind of hack.

try:
    mhou_fit = fit.as_input()["theorycovmatconfig"]["use_user_uncertainties"]
    if mhou_fit:
        mhou_covmat = API.user_covmat(**(inps_central_fit | fit.as_input()["theorycovmatconfig"]))
        exp_covmat = C  # we don't use exp_covmat, but may be useful to keep
        C = C + mhou_covmat
except:
    pass

In [23]:
# Note this is NOT the same as the prediction of the mean PDF (i.e. replica0)
mean_prediction = theorypreds_fit.mean(axis=1)

X = np.zeros_like(C.values)
for i in range(theorypreds_fit.shape[1]):
    X += np.outer(
        (theorypreds_fit.iloc[:, i] - mean_prediction),
        (theorypreds_fit.iloc[:, i] - mean_prediction),
    )
X *= 1 / theorypreds_fit.shape[1]

In [24]:
pseudodata = API.read_pdf_pseudodata(**common_dict)

In [25]:
# We should use <D>_rep instead of D_exp, though if we do the sampling
# faithfully the central value over replicas should be the same as the
# experimental central value. For this set resample_negative_pseudodata=False in
# the n3fit runcard.
dat_reps = pd.concat(
    [i.pseudodata.reindex(prior_theorypreds_central.index) for i in pseudodata],
    axis=1,
)

In [ ]:
invcov = np.linalg.inv(C + S)
# delta_T_tilde is Eq. 3.37 in https://arxiv.org/pdf/2105.05114
delta_T_tilde = -S_hat @ invcov @ (mean_prediction - dat_reps.mean(axis=1))

# P_tilde is Eq. 3.38.
#
# Note that not all terms of the equation in the paper are here, in particular
# X_tile and X_hat vanish. This is because they measure the covariance of
# T_tilde over PDF replicas, but for us T_tilde is alphas. The prediciton of
# alphas does not depend on the PDF, and as such T_tilde^(r) == T_tilde^(0)
P_tilde = S_hat.T @ invcov @ X @ invcov @ S_hat + S_tilde - S_hat.T @ invcov @ S_hat

pred = alphas_central + delta_T_tilde
unc = np.sqrt(P_tilde)
print(rf"Prediction for $\alpha_s(M_Z)$: {pred:.4f} ± {unc:.4f}")